In [1]:
import pandas as pd
import numpy as n
import math

In [2]:
workload = pd.read_csv(r"raw_size.csv")
workload

,site,domain,env,host_profile,name,type,count,vcpu,ram_gb,estorage_gb,pstorage_gb
0,Site 1,Prod,Infra,vRPA,vRPA Addtl,VM,2,8,32,0,35
1,Site 1,Prod,Infra,NSX,NSX-Edge,VM,2,16,64,50,200
2,Site 1,Prod,Prod,K8s-Master,K8s-Master,VM,3,4,16,2,50
3,Site 1,Prod,Prod,K8s-Worker-Apps,ACME API,Pod,20,4,16,2,200
4,Site 1,Prod,Prod,K8s-Worker-Apps,ACME Backend API,Pod,12,4,16,2,200
...,...,...,...,...,...,...,...,...,...,...,...
74,Site 2,DR,DR,K8s-Worker-DS,Foo Redis,Pod,3,4,16,2,200
75,Site 2,DR,DR,K8s-Worker-DB1,ACME Database,Pod,3,16,64,2,2000
76,Site 2,DR,DR,K8s-Worker-DB2,Foo Database,Pod,3,8,16,2,200
77,Site 2,DR,DR,K8s-Worker-Apps,SMS Gateway,Pod,5,4,16,2,200


In [3]:
workload['total_vcpu'] = workload['count'] * workload['vcpu']
workload['total_ram_gb'] = workload['count'] * workload['ram_gb']
workload['total_storage_gb'] = workload['count'] * (workload['estorage_gb'] + workload['pstorage_gb'])
workload

,site,domain,env,host_profile,name,type,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
0,Site 1,Prod,Infra,vRPA,vRPA Addtl,VM,2,8,32,0,35,16,64,70
1,Site 1,Prod,Infra,NSX,NSX-Edge,VM,2,16,64,50,200,32,128,500
2,Site 1,Prod,Prod,K8s-Master,K8s-Master,VM,3,4,16,2,50,12,48,156
3,Site 1,Prod,Prod,K8s-Worker-Apps,ACME API,Pod,20,4,16,2,200,80,320,4040
4,Site 1,Prod,Prod,K8s-Worker-Apps,ACME Backend API,Pod,12,4,16,2,200,48,192,2424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Site 2,DR,DR,K8s-Worker-DS,Foo Redis,Pod,3,4,16,2,200,12,48,606
75,Site 2,DR,DR,K8s-Worker-DB1,ACME Database,Pod,3,16,64,2,2000,48,192,6006
76,Site 2,DR,DR,K8s-Worker-DB2,Foo Database,Pod,3,8,16,2,200,24,48,606
77,Site 2,DR,DR,K8s-Worker-Apps,SMS Gateway,Pod,5,4,16,2,200,20,80,1010


In [4]:
pods = workload.loc[workload['type'] == "Pod"]
pods

,site,domain,env,host_profile,name,type,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
3,Site 1,Prod,Prod,K8s-Worker-Apps,ACME API,Pod,20,4,16,2,200,80,320,4040
4,Site 1,Prod,Prod,K8s-Worker-Apps,ACME Backend API,Pod,12,4,16,2,200,48,192,2424
5,Site 1,Prod,Prod,K8s-Worker-DS,ACME Eventbus,Pod,5,4,16,2,200,20,80,1010
6,Site 1,Prod,Prod,K8s-Worker-Apps,Foo SSO API,Pod,15,4,16,2,200,60,240,3030
7,Site 1,Prod,Prod,K8s-Worker-Apps,Foo Account API,Pod,15,4,16,2,200,60,240,3030
8,Site 1,Prod,Prod,K8s-Worker-Apps,Foo Kong API,Pod,3,4,16,2,200,12,48,606
9,Site 1,Prod,Prod,K8s-Worker-DS,Foo Elasticsearch,Pod,3,4,16,2,200,12,48,606
10,Site 1,Prod,Prod,K8s-Worker-DS,Foo RabbitMQ,Pod,3,4,16,2,200,12,48,606
11,Site 1,Prod,Prod,K8s-Worker-DS,Foo Redis,Pod,3,4,16,2,200,12,48,606
12,Site 1,Prod,Prod,K8s-Worker-DB1,ACME Database,Pod,3,16,64,2,2000,48,192,6006


In [5]:
vms = workload.loc[workload['type'] == "VM"]
vms

,site,domain,env,host_profile,name,type,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
0,Site 1,Prod,Infra,vRPA,vRPA Addtl,VM,2,8,32,0,35,16,64,70
1,Site 1,Prod,Infra,NSX,NSX-Edge,VM,2,16,64,50,200,32,128,500
2,Site 1,Prod,Prod,K8s-Master,K8s-Master,VM,3,4,16,2,50,12,48,156
15,Site 1,Prod,Prod-Reporting,Reporting Server,Reporting Server,VM,1,8,64,20,600,8,64,620
16,Site 1,Non-Prod,Management,Other,Other,VM,15,6,18,0,256,90,270,3840
17,Site 1,Non-Prod,Management,NSX,NSX Controller,VM,8,8,32,0,200,64,256,1600
18,Site 1,Non-Prod,Infra,NSX,NSX-Edge,VM,2,16,64,50,200,32,128,500
19,Site 1,Non-Prod,Pre-Prod,K8s-Master,K8s-Master,VM,3,4,16,2,50,12,48,156
32,Site 1,Non-Prod,Pre-Prod,Reporting Server,Reporting Server,VM,1,2,16,20,20,2,16,40
33,Site 1,Non-Prod,Staging,K8s-Master,K8s-Master,VM,3,4,16,2,50,12,48,156


In [16]:
pod_summary = pods.groupby(["site"])
pod_summary

AttributeError: 'str' object has no attribute 'ndim'

In [ ]:
workers = []
for name, group in pod_summary:
   site = name[0]
   domain = name[1]
   env = name[2]
   host_profile = name[3]
   worker_name = name[3]
   worker_type = "VM"
   
   pods = group.sum()   

   pods_per_worker = math.floor(math.sqrt(pods['count']))
   worker_name = worker_name + " PPW(" + str(pods_per_worker) + ")"

   min_workers = math.ceil(pods['count']/pods_per_worker)
   
   # 2 vcpu for system service, 1.2 time (or 20% extra) for spare capacity
   min_worker_cpu = math.ceil( ( ( pods['total_vcpu']/min_workers ) + 2 ) * 1.2)
   
   # 4 gb for system service, 1.2 time (or 20% extra) for spare capacity
   min_worker_ram = math.ceil( ( ( pods['total_ram_gb']/min_workers ) + 4 ) * 1.2 )

   min_worker_estorage = 50
   min_worker_pstorage = pods['total_storage_gb']/min_workers
   
   worker_count = min_workers + 3
   worker_vcpu = min_worker_cpu
   worker_ram_gb = min_worker_ram
   worker_estorage_gb = math.ceil(min_worker_estorage)
   worker_pstorage_gb = math.ceil(min_worker_pstorage)
   total_worker_vcpu = worker_count * worker_vcpu
   total_worker_ram_gb = worker_count * worker_ram_gb
   total_worker_storage_gb = worker_count * (worker_estorage_gb + worker_pstorage_gb)

   workers.append([site,domain,env,host_profile, worker_name, worker_type, worker_count, worker_vcpu, worker_ram_gb, worker_estorage_gb, worker_pstorage_gb, total_worker_vcpu, total_worker_ram_gb, total_worker_storage_gb])

In [13]:
workers_df = pd.DataFrame(data=workers, columns=workload.columns)
workers_df

,site,domain,env,host_profile,name,type,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
0,Site 1,Non-Prod,Dev,K8s-Worker-Apps,K8s-Worker-Apps PPW(2),VM,6,6,14,50,44,36,84,564
1,Site 1,Non-Prod,Dev,K8s-Worker-DB1,K8s-Worker-DB1 PPW(1),VM,4,5,15,50,22,20,60,288
2,Site 1,Non-Prod,Dev,K8s-Worker-DB2,K8s-Worker-DB2 PPW(1),VM,4,5,15,50,22,20,60,288
3,Site 1,Non-Prod,Dev,K8s-Worker-DS,K8s-Worker-DS PPW(2),VM,5,6,18,50,44,30,90,470
4,Site 1,Non-Prod,Pre-Prod,K8s-Worker-Apps,K8s-Worker-Apps PPW(3),VM,7,11,32,50,66,77,224,812
5,Site 1,Non-Prod,Pre-Prod,K8s-Worker-DB1,K8s-Worker-DB1 PPW(1),VM,6,5,15,50,22,30,90,432
6,Site 1,Non-Prod,Pre-Prod,K8s-Worker-DB2,K8s-Worker-DB2 PPW(1),VM,6,5,15,50,22,30,90,432
7,Site 1,Non-Prod,Pre-Prod,K8s-Worker-DS,K8s-Worker-DS PPW(3),VM,7,14,45,50,66,98,315,812
8,Site 1,Non-Prod,Staging,K8s-Worker-Apps,K8s-Worker-Apps PPW(2),VM,6,8,23,50,44,48,138,564
9,Site 1,Non-Prod,Staging,K8s-Worker-DB1,K8s-Worker-DB1 PPW(1),VM,4,5,15,50,22,20,60,288


In [14]:
site = vms.append(workers_df).sort_values(["site","domain", "env", "host_profile", "name"])
site

,site,domain,env,host_profile,name,type,count,vcpu,ram_gb,estorage_gb,pstorage_gb,total_vcpu,total_ram_gb,total_storage_gb
47,Site 1,Non-Prod,Dev,K8s-Master,K8s-Master,VM,3,4,16,2,50,12,48,156
0,Site 1,Non-Prod,Dev,K8s-Worker-Apps,K8s-Worker-Apps PPW(2),VM,6,6,14,50,44,36,84,564
1,Site 1,Non-Prod,Dev,K8s-Worker-DB1,K8s-Worker-DB1 PPW(1),VM,4,5,15,50,22,20,60,288
2,Site 1,Non-Prod,Dev,K8s-Worker-DB2,K8s-Worker-DB2 PPW(1),VM,4,5,15,50,22,20,60,288
3,Site 1,Non-Prod,Dev,K8s-Worker-DS,K8s-Worker-DS PPW(2),VM,5,6,18,50,44,30,90,470
60,Site 1,Non-Prod,Dev,Reporting Server,Reporting Server,VM,1,2,16,20,20,2,16,40
18,Site 1,Non-Prod,Infra,NSX,NSX-Edge,VM,2,16,64,50,200,32,128,500
17,Site 1,Non-Prod,Management,NSX,NSX Controller,VM,8,8,32,0,200,64,256,1600
16,Site 1,Non-Prod,Management,Other,Other,VM,15,6,18,0,256,90,270,3840
19,Site 1,Non-Prod,Pre-Prod,K8s-Master,K8s-Master,VM,3,4,16,2,50,12,48,156


In [15]:
site.groupby(["site","domain"]).sum()


count  vcpu  ram_gb  estorage_gb  pstorage_gb  total_vcpu  \
site   domain                                                                
Site 1 Non-Prod    101   133     464          716         1306         707   
       Prod         40   126     496          272         5227         880   
Site 2 DR           63   140     546          272         5833        1034   

                 total_ram_gb  total_storage_gb  
site   domain                                    
Site 1 Non-Prod          2277             12236  
       Prod              3272             39562  
Site 2 DR                3798             45452